In [ ]:
import os
import numpy as np
import pandas as pd

# pandapower imports
import pandapower as pp
import pandapower.plotting as plot
import matplotlib.pyplot as plt

from pandapower.timeseries import DFData, OutputWriter
from pandapower.control import ConstControl

from pandapower.timeseries.run_time_series import run_timeseries
#from pandaplan.core.timeseries.run_profile_cython.run_cython import run_timeseries

import simbench

from sklearn.model_selection import train_test_split

# torch imports
import torch
from torch_geometric.data import Data

#
from pandapowerHelpers.simbench_helpers import (
    create_gen_profile_df_from_simbench
)
from pandapowerHelpers.toolbox_extension import create_unique_ids
from pandapowerHelpers import (
    simple_plot_custom,
    plot_powerflow_quiver,
    plot_capacities,
    write_line_annotations,
    write_bus_annotations
)

plt.rcParams.update({'font.size': 12})

Let's grab the network and have a look
--------

This is "Case 5" from the power system test cases. The controllable generators have been replaced by static generators. 

In [ ]:
net = pp.from_json(os.path.join(pp.pp_dir, "networks","power_system_test_case_jsons", "case5.json"))
#net.sgen = pd.concat([net.gen,net.sgen])
net.sgen = net.gen[:]
net.sgen['q_mvar'] = 0
net.gen = net.gen[0:0]
create_unique_ids(net,'load')
create_unique_ids(net,'sgen')
pp.runpp(net)

In [ ]:
net.load

In [ ]:
bus_size=0.03
text_size=30

In [ ]:
net.res_line

In [ ]:
#plot.simple_plot(net, show_plot=True)
fig,axes = plt.subplots(1,1,figsize=(8,8))

ax = axes
simple_plot_custom(net,ax=ax)
plot_capacities(net,ax=ax,size=bus_size);
plot_powerflow_quiver(net,ax=ax,scale=1,units='x',pivot='mid',width=0.025,zorder=100,n_arrows_smallestline=2)

# Some line loading annotations
lineid_loading = np.char.mod('%.0f%%',net.res_line['loading_percent'])
#write_line_annotations(net,ax,net.line.index,lineid_loading,size=text_size,margin=0.08)
line_index = np.char.mod('line %d',net.line.index)
write_line_annotations(net,ax,net.line.index,line_index,size=text_size,margin=0.08)

load_bus_index = net.load.groupby('bus').sum().index
load_p = np.char.mod('$\minus$%.0f MW',net.load.groupby('bus').sum()['p_mw'])
gen_bus_index = net.gen.groupby('bus').sum().index
gen_p  = np.char.mod('+%.0f MW',net.gen.groupby('bus').sum()['p_mw'])

bus_index = np.char.mod('bus %d',net.bus.index)
write_bus_annotations(net,ax,net.bus.index,bus_index,size=text_size,margin=0.01,placement='topleft')
#write_bus_annotations(net,ax,load_bus_index,load_p,size=0.1,margin=0.01,placement='topright')
#write_bus_annotations(net,ax,gen_bus_index,gen_p,size=0.1,margin=0.01,placement='bottomright')
ax.set_xlim([-4.5,-2.0])#ax = axes[1]
#simple_plot_custom(net,ax=ax,use_bus_geodata=True);
#plot_powerflow_quiver(net,ax=ax,scale=1,units='x',pivot='mid',width=0.020,zorder=100,n_arrows_smallestline=3)

Load some Simbench profiles and look at the network
--------

Load and generator profiles are taken from the list of simbench profiles.

In [ ]:
profiles = simbench.get_all_simbench_profiles(0)

In [ ]:
def create_load_profile_df_from_simbench(load_df,_profiles,profile_name_list=[],isQ=False) :
    # load_df expects "net.load"
    # If no profile_name_list is provided, then
    # the function will assume that a 'profile' column exists in `net`.

    profile_dict = {}

    for i,load_row in load_df.iterrows() :

        # Get the column name (e.g. "BL-H_qload")
        profilename = profile_name_list[i] if len(profile_name_list) else load_row['profile']
        columnname = '{}_{}load'.format(profilename,'q' if isQ else 'p')

        # Scale by the initial net load (standard in simbench)
        scale_factor = load_row.q_mvar if isQ else load_row.p_mw

        # Find the profile
        profile = _profiles['load'][columnname]

        # Scale the profile
        profile_dict[load_row['name']] = scale_factor * profile

    profile_df = pd.DataFrame.from_dict(profile_dict)
    return profile_df

In [ ]:
# Grab profiles from Simbench

load_profile_name_list = ['G3-H','H0-H','L2-M']
sgen_profile_name_list = ['PV2','pp_1','WP1']

max_timestep = 35136

load_df_p = create_load_profile_df_from_simbench(net.load,profiles,profile_name_list=load_profile_name_list,isQ=False)
load_df_q = create_load_profile_df_from_simbench(net.load,profiles,profile_name_list=load_profile_name_list,isQ=True)
sgen_df_p  = create_gen_profile_df_from_simbench(net.sgen,profiles,profile_name_list=sgen_profile_name_list)

load_df_p = load_df_p[:max_timestep]
load_df_q = load_df_q[:max_timestep]
sgen_df_p = sgen_df_p[:max_timestep]

# Scale up the PV to make it more consequential
sgen_df_p['sgen_bus00_01'] = sgen_df_p['sgen_bus00_01']*20

In [ ]:
fig,axes = plt.subplots(1,1,figsize=(16,4))
ax = axes

max_timestep_plot = 35136

load_labels = list('{} {}'.format(row['name'],load_profile_name_list[i]) for i,row in net.load.iterrows())
sgen_labels = list('{} {}'.format(row['name'],sgen_profile_name_list[i] ) for i,row in net.sgen.iterrows())

ax.plot(load_df_p[:max_timestep_plot],label=load_labels)
ax.plot(sgen_df_p[:max_timestep_plot],label=sgen_labels)
ax.set(xlabel='timestep',ylabel='Power [MW]')
ax.legend(ncol=2);
ax.set_ylim([0,700])

In [ ]:
# Load profiles into net

dfdata_load_p = DFData(load_df_p)
dfdata_load_q = DFData(load_df_q)
dfdata_sgen_p = DFData(sgen_df_p)

net.controller = net.controller[0:0]
ConstControl(net, element='load', variable='p_mw', element_index=net.load.index,
             data_source=dfdata_load_p, profile_name=net.load['name'])
ConstControl(net, element='load', variable='q_mvar', element_index=net.load.index,
             data_source=dfdata_load_q, profile_name=net.load['name'])
ConstControl(net, element='sgen', variable='p_mw', element_index=net.sgen.index,
             data_source=dfdata_sgen_p, profile_name=net.sgen['name'])

The input data X
--------

In [ ]:
# The shape of the X data should be (n_timesteps, n_inputs)
X = pd.concat([load_df_p,load_df_q,sgen_df_p],axis=1)
X = X.values
X.shape

In [ ]:
load_df_p_csv = load_df_p.rename(columns={i:'{}_p'.format(i) for i in load_df_p.columns})
load_df_q_csv = load_df_q.rename(columns={i:'{}_q'.format(i) for i in load_df_q.columns})
sgen_df_p_csv = sgen_df_p.rename(columns={i:'{}_p'.format(i) for i in sgen_df_p.columns})

In [ ]:
X_dataframe = pd.concat([load_df_p_csv,load_df_q_csv,sgen_df_p_csv],axis=1)
X_dataframe

In [ ]:
net.load

The output data Y
---------

In [ ]:
# the outputwriter contains the power flow results
if hasattr(net,'output_writer') :
    del net.output_writer

run_timeseries(net, time_steps=range(max_timestep), verbose=False)
Y = net.output_writer.loc[0, "object"].output
Y = Y['res_line.loading_percent'].values

Y.shape

Save the input and output to a csv file that can be loaded for the exercise.
--------

In [ ]:
Y_dataframe = net.output_writer.loc[0, "object"].output['res_line.loading_percent']
if 'loading_percent' not in str(Y_dataframe.columns[0]) :
    Y_dataframe = Y_dataframe.rename(columns={i:'line {} loading_percent'.format(i) for i in Y_dataframe.columns})

In [ ]:
Y_dataframe

In [ ]:
all_data = pd.concat([X_dataframe,Y_dataframe],axis=1)
all_data.to_csv('case5.csv',index=False)

all_data

For the analysis, see "grid_powerflow_with_ann"
-------